In [2]:
import pandas as pd
import sys
if '..' not in sys.path:
    sys.path.append('..')

print(sys.path)

df = pd.read_pickle('data/df_heartRate.pkl')

['/Users/mnann/Documents/Code/PulseFi/src/data', '/Users/mnann/.pyenv/versions/3.8.13/lib/python38.zip', '/Users/mnann/.pyenv/versions/3.8.13/lib/python3.8', '/Users/mnann/.pyenv/versions/3.8.13/lib/python3.8/lib-dynload', '', '/Users/mnann/Documents/Code/PulseFi/venv/lib/python3.8/site-packages', '..']


FileNotFoundError: [Errno 2] No such file or directory: '/data/df_heartRate.pkl'

In [3]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth
from datetime import datetime

scope = "user-read-recently-played"

sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope, client_id=client_id, client_secret=client_secret, redirect_uri="http://localhost:8888/callback"))
# results = sp.current_user_recently_played()
results = sp.current_user_recently_played()

df_played = pd.DataFrame(results['items'])
# df_played['played_at'] = df_played['played_at'].apply(lambda x: datetime.strptime(x, '%Y-%m-%dT%H:%M:%S.%fZ'))
# df_played['start_time'] = pd.to_datetime(df_played['played_at'])
df_played['name'] = df_played['track'].apply(lambda x: x['name'])
df_played['artist'] = df_played['track'].apply(lambda x: x['artists'][0]['name'])
df_played['song_id'] = df_played['track'].apply(lambda x: x['id']) 
df_played['duration_ms'] = df_played['track'].apply(lambda x: x['duration_ms'])
df_played['songImage'] = df_played['track'].apply(lambda x: x['album']['images'][0]['url'])
# df_played['end_time'] = df_played['played_at'] + pd.to_timedelta(df_played['track'].apply(lambda x: x['duration_ms']), unit='ms')
# display(df_played)

# results = None
# for i in range(10):
#     if i == 0:
#         results = sp.current_user_recently_played()
#     else:
#         print(results)
#         results = sp.current_user_recently_played(after=results['cursors']['after'])
#     for i, item in enumerate(results['items']):
#         track = item['track']
#         secondsPlayed = track['duration_ms']/1000
#         timePlayed = datetime.fromtimestamp(secondsPlayed).strftime('%M:%S')
#         played_at = datetime.strptime(item['played_at'], '%Y-%m-%dT%H:%M:%S.%fZ')
#         print(str(played_at) + track['name'] + ' - ' + track['artists'][0]['name'] + ' - ' + ' - ' + timePlayed)


In [50]:
from oura.v2 import OuraClientV2, OuraClientDataFrameV2
import requests 
from datetime import datetime, timedelta
import pandas as pd

import sys
sys.path.append('..')

from __init__ import getSecret


class OuraData(): 

    def __init__(self):
        '''
        TODO
        '''

        self.oura_headers = { 
            'Authorization': 'Bearer ' + getSecret('OURA_KEY') 
        }

    def ouraParams(self):
        endDate = datetime.now()
        startDate = endDate - timedelta(days=10)
        # params={ 
        #     'start_datetime': datetime.strptime('2022-01-01', '%Y-%m-%d').strftime('%Y-%m-%d %H:%M:%S'),
        #     'end_datetime': (datetime.now() - timedelta(days=-10)).strftime('%Y-%m-%d %H:%M:%S'),
        # }
        params={
            'start_datetime': startDate.strftime('%Y-%m-%d %H:%M:%S'),
            'end_datetime': endDate.strftime('%Y-%m-%d %H:%M:%S'),
        }
        return params
    
    # def heartRate(self, startDate=None, endDate=None):
    #     '''
    #     TODO
    #     '''
    #     ouraClient_df = OuraClientDataFrameV2(personal_access_token=getSecret('OURA_KEY'))  
    #     if startDate is None or endDate is None:
    #         raw = ouraClient_df.heartrate()
    #     else:
    #         raw = ouraClient_df.heartrate(start_date=startDate, end_date =endDate)
    #         # raw = ouraClient_df.heartrate(
    #     df_hr = pd.DataFrame(raw['data'])
    #     df_hr['timestamp'] = pd.to_datetime(df_hr['timestamp'])
    #     return df_hr

    def apiFunctions(self):

        ouraClient = OuraClientV2(personal_access_token=getSecret('OURA_KEY'))
        ouraClient_df = OuraClientDataFrameV2(personal_access_token=getSecret('OURA_KEY'))

        # functions available in the API
        daily_activity = ouraClient_df.daily_activity()
        heart_rate_sleepDaily = ouraClient_df.heart_rate_df()
        heartrate = ouraClient_df.heartrate()
        personal_info = ouraClient_df.personal_info()
        personal_info_df = ouraClient_df.personal_info_df()
        session = ouraClient_df.session()
        sessions_df = ouraClient_df.sessions_df()
        # functions that haven't been implemented in the api
        # sleep_df = ouraClient_df.sleep_df()
        # tags = ouraClient_df.tags()
        # tags_df = ouraClient_df.tags_df()
        # workouts = ouraClient_df.workouts()
        # workouts_df = ouraClient_df.workouts_df()
        # readiness_df = ouraClient_df.readiness_df()

        t1 = ouraClient.daily_activity()
        t2 = ouraClient.heartrate()
        t3 = ouraClient.personal_info()
        t4 = ouraClient.session()
        t5 = ouraClient.tags()
        t6 = ouraClient.workouts()

    def dailySleep(self):
        url = 'https://api.ouraring.com/v2/usercollection/daily_sleep' 
        response = requests.request('GET', url, headers=self.oura_headers, params=self.ouraParams()) 
        sleepDaily = pd.read_json(response.text)
        sleepDaily = pd.concat([sleepDaily.drop(['data'], axis=1), sleepDaily['data'].apply(pd.Series)], axis=1)
        sleepDaily = pd.concat([sleepDaily.drop(['contributors'], axis=1), sleepDaily['contributors'].apply(pd.Series)], axis=1)
        sleepDaily.drop(['next_token','day'], inplace=True, axis=1)
        sleepDaily['day'] = pd.to_datetime(sleepDaily['timestamp']).dt.tz_convert('UTC').dt.date
        # sleepDaily.drop(['next_token','day','timestamp'], inplace=True, axis=1)
        sleepDaily.rename(columns={'score':'sleepScore'}, inplace=True)
        return sleepDaily
    def detailedSleep(self):
        url = 'https://api.ouraring.com/v2/usercollection/sleep' 
        response = requests.request('GET', url, headers=self.oura_headers, params=self.ouraParams()) 
        df = pd.read_json(response.text)
        df = pd.concat([df.drop(['data'], axis=1), df['data'].apply(pd.Series)], axis=1)
        df.drop(['next_token'],inplace=True,axis=1)
        df = pd.concat([df.drop(['readiness'], axis=1), df['readiness'].apply(pd.Series, dtype='object')], axis=1)
        df.rename(columns={'score': 'readiness_score'}, inplace=True)
        df['day'] = pd.to_datetime(df['day']).dt.date
        return df
    def nightlySleep(self):
        sleepDaily = self.dailySleep()
        sleepData = self.detailedSleep()
        sleepData = sleepData.loc[sleepData['type'] == 'long_sleep']
        sleepData = sleepData.merge(sleepDaily, how='left', on='day')
        return sleepData
    def heartRate(self):
        url = 'https://api.ouraring.com/v2/usercollection/heartrate'
        response = requests.request('GET', url, headers=self.oura_headers, params=self.ouraParams())
        # if 'detail' in response.text and response.text['detail'] == "Timerange between start and endtime has to be less than 30 days":
        #     print("Timerange between start and endtime has to be less than 30 days")
        #     return None
        df = pd.read_json(response.text)
        df = pd.concat([df.drop(['data'], axis=1), df['data'].apply(pd.Series)], axis=1)
        return df

    def saveData(self):
        df_download = pd.read_csv('oura_2022-01-17_2022-12-21_trends.csv')

# today
# endDate = (datetime.now() - timedelta(days=39)).strftime('%Y-%m-%d')
# # 7 days ago
# startDate = (datetime.now() - timedelta(days=40)).strftime('%Y-%m-%d')
# print(startDate, endDate)
df_hr = OuraData().heartRate() #startDate=startDate, endDate=endDate)
display(df_hr)

,next_token,bpm,source,timestamp
0,NaN,78,awake,2023-03-31T14:53:33+00:00
1,NaN,69,awake,2023-03-31T14:53:46+00:00
2,NaN,76,awake,2023-03-31T14:53:53+00:00
3,NaN,62,awake,2023-03-31T15:34:03+00:00
4,NaN,115,awake,2023-03-31T15:34:17+00:00
...,...,...,...,...
1901,NaN,64,awake,2023-04-10T03:28:56+00:00
1902,NaN,54,awake,2023-04-10T03:29:05+00:00
1903,NaN,47,awake,2023-04-10T13:57:43+00:00
1904,NaN,47,awake,2023-04-10T13:58:02+00:00


In [51]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go


fig = go.Figure()
fig.add_trace(go.Scatter(x=df_hr['timestamp'], y=df_hr['bpm'], name='resting_heart_rate'))